In [ ]:
import pandas as pd
import numpy as np
import requests
import re
import time
from bs4 import BeautifulSoup
from tqdm.notebook import tqdm 
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options

In [167]:
chrome_options = webdriver.ChromeOptions()
chrome_path = "C:\Program Files\Google\Chrome\Application\chrome.exe"  # 실제 경로
chrome_options.binary_location = chrome_path

driver_path = "chromedriver.exe"
myservice = Service(driver_path)
driver = webdriver.Chrome(service=myservice, options=chrome_options)

print(type(driver))

wait_time = 10
driver.implicitly_wait(wait_time)

driver.maximize_window()

<class 'selenium.webdriver.chrome.webdriver.WebDriver'>


In [168]:
# 제목 / 작가 / 평점 / 이미지파일 / 링크
weekdays = ['mon', 'tue', 'wed', 'thu', 'fri', 'sat', 'sun']
base_url = "https://comic.naver.com/webtoon?tab="
weekly_webtoons = {}

for weekday in weekdays:
    page_url = f"{base_url}{weekday}"
    driver.get(page_url)
    time.sleep(3)
    
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    
    title_tags = soup.find_all("span", class_="ContentTitle__title--e3qXt")
    author_tags = soup.find_all("a", class_="ContentAuthor__author--CTAAP")
    star_tags = soup.find_all("span", class_="Rating__star_area--dFzsb")
    img_tags = soup.find_all("img", class_="Poster__image--d9XTI")
    link_tags = soup.select("a.Poster__link--sopnC")

    webtoon_data = []
    for idx in range(3, len(title_tags)):
        title = title_tags[idx].get_text(strip=True)
        author = author_tags[idx].get_text(strip=True) if idx < len(author_tags) else ""
        star = star_tags[idx].get_text(strip=True)[2:] if idx < len(star_tags) else ""

        # 이미지 URL 추출
        img_url = img_tags[idx]['src'] if idx < len(img_tags) else ""

        # 링크 추출
        link_url = "https://comic.naver.com" + link_tags[idx].get("href") if idx < len(link_tags) else ""

        webtoon_data.append([title, author, star, img_url, link_url])
        
    weekly_webtoons[weekday] = webtoon_data[:10]
    print(f"{weekday} 요일 {len(webtoon_data[:10])}개 수집 완료\n")

driver.quit()


mon 요일 10개 수집 완료

tue 요일 10개 수집 완료

wed 요일 10개 수집 완료

thu 요일 10개 수집 완료

fri 요일 10개 수집 완료

sat 요일 10개 수집 완료

sun 요일 10개 수집 완료



In [169]:
day_ko = {'mon': '월', 'tue': '화', 'wed': '수', 'thu': '목', 'fri': '금', 'sat': '토', 'sun': '일'}

df = pd.DataFrame([
    (day_ko[day], title, author, star, img, link)
    for day, webtoons in weekly_webtoons.items()
    for title, author, star, img, link in webtoons
], columns=['요일', '제목', '작가', '평점', '이미지', '링크'])

df.to_csv('weekly_webtoons.csv', index=False, encoding='utf-8-sig')
print("저장 완료")


저장 완료
